In [1]:
import os
import os.path as osp
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2
import shutil

In [2]:
def make_dirs(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

In [11]:
## 对百度地址识别结果进行预标注
baidu_images_root = '/mnt/data/rz/data/idCard/v5/rec/address/baidu/images'
baiduLabelp = '/mnt/data/rz/data/idCard/v5/rec/address/baidu/baiduLabels.txt'
custom_root = '/mnt/data/rz/programe/KREP/idCard/data/custom/address'
baidu_items_labelsp = './data/custom/baidu_items_labels.txt'

labeled_items_store_root = '/mnt/data/rz/data/idCard/v5/rec/address/process'
labeled_items_store_img = osp.join(labeled_items_store_root, 'images') # 用百度识别结果标注的字段图片存储地址
labeled_items_label_path = osp.join(labeled_items_store_root, 'itemLabels.txt') # 用百度识别结果标注的字段内容文件
error_store_root = osp.join(labeled_items_store_root, 'error') # 自识别的结果与百度的结果没有匹配上的时候，转人工,将错误的图片与标注文件存到此处
error_store_baidu = osp.join(error_store_root, 'baidu')
error_store_custom = osp.join(error_store_root, 'custom')



if not os.path.exists(error_store_root):
    os.makedirs(error_store_root)
if not os.path.exists(labeled_items_store_root):
    os.makedirs(labeled_items_store_root)
if not os.path.exists(labeled_items_store_img):
    os.makedirs(labeled_items_store_img)
make_dirs(error_store_root)


In [4]:
custom_address_imgp = glob(osp.join(custom_root, '*.jpg'))
custom_address_items_info = dict() # 一个地址地段图片中有多行，裁切出多个行文本图片

for cus_imgp in custom_address_imgp:
    imgn = osp.basename(cus_imgp)
    name = imgn.replace('.jpg', '')
    y1, suffix = name.split('_')
    if not suffix in custom_address_items_info:
        custom_address_items_info[suffix] = {'img':[imgn], 'txt':[name + '.txt'], 'Y':[int(y1)]}
    else:
        custom_address_items_info[suffix]['img'].append(imgn)
        custom_address_items_info[suffix]['txt'].append(name+'.txt')
        custom_address_items_info[suffix]['Y'].append(int(y1))

In [27]:
def store_custom_error_data(org_root, dst_root, data):
    lines = []
    if not osp.exists(dst_root):
        os.makedirs(dst_root)
    dst_img_root = osp.join(dst_root, 'images')
    if not osp.exists(dst_img_root):
        os.makedirs(dst_img_root)
    for item in data:
        imgs = item['img']
        txts = item['txt']
        for imgn, txt in zip(imgs, txts):
            imgp = osp.join(org_root, imgn)
            shutil.copy(imgp, dst_root)
            line = imgn + '\t' + txt
            lines.append(line)
    if len(lines) > 0:
        labelp = osp.join(dst_root, 'label.txt')
        with open(labelp, 'w') as f:
            f.write('\n'.join(lines))
    
def store_baidu_error_data(org_root, dst_root, data):
    lines = []
    if not osp.exists(dst_root):
        os.makedirs(dst_root)
    dst_img_root = osp.join(dst_root, 'images')
    if not osp.exists(dst_img_root):
        os.makedirs(dst_img_root)
    for line in data:
        imgn, txt = line.strip().split('\t')
        imgp = osp.join(org_root, imgn)
        shutil.copy(imgp, dst_img_root)
        lines.append(line)
    if len(lines) > 0:
        labelp = osp.join(dst_root, 'label.txt')
        with open(labelp, 'w') as f:
            f.write('\n'.join(lines))


In [28]:
# 百度地址字段得识别结果(百度地址字段得位置是一整块得，没有进行字段切分)
baidu_error = []
custom_error = []
baidu_items_labels = []
i = 0
with open(baiduLabelp) as f:
    for line in f:
        bimgn, baddress = line.strip().split('\t')
        suffix = bimgn.replace('.jpg', '')
        custom_info = custom_address_items_info.get(suffix, None)
        if custom_info is not None:
            y = custom_info['Y']
            txt_names = custom_info['txt']
            img_names = custom_info['img']
            custom_address = []
            baidu_address = [] # 用百度的识别结果替换后的地址字段结果
            custom_address_place = dict()
            idx = np.argsort(y)
            for txtn in txt_names:
                txtp = osp.join(custom_root, txtn)
                with open(txtp) as f:
                    caddress = f.read().strip().split('\t')[1]
                    custom_address.append(caddress)
            len_address = len(''.join(custom_address)) # 地址的总长度
            # 当custom的地址字段的字符数与百度地址字段的字符数相同，则用百度的结果去替换它
            if len_address == len(baddress):
                begin = 0
                end = 0
                for ix in idx:
                    end = min(begin + len(custom_address[ix]), len_address)
                    baidu_item_address = baddress[begin:end]
                    imgn = img_names[ix]
                    line = imgn + '\t' + baidu_item_address
                    begin = end
                    baidu_items_labels.append(line)
                    # im = cv2.imread(osp.join(custom_root, imgn))
                    shutil.copy(osp.join(custom_root, imgn), labeled_items_store_img)
                    # plt.imshow(im)
                    # plt.show()
            else:
                custom_error.append(custom_info)
        else:
            baidu_error.append(line.strip())
        # i += 1
        # if i == 10:
        #     break
    # 用百度识别结果替换后的字段label文件
    with open(labeled_items_label_path, 'w') as f:
        f.write('\n'.join(baidu_items_labels))
    # 存储没有被切分的整块地址字段图片信息
    store_baidu_error_data(baidu_images_root, error_store_baidu, baidu_error)
    store_custom_error_data(custom_root, error_store_custom, custom_error)
